# Import dataset

In [15]:
# importing libraries
import pandas as pd
import scipy
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt


In [16]:
df = pd.read_csv('hotel_bookings_data.csv')
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_weekdays_nights,adults,...,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
0,Resort Hotel,0,342,2017,September,27,1,0,0,2,...,3,No Deposit,NaN,NaN,0,Personal,0.0,0,0,Check-Out
1,Resort Hotel,0,737,2017,September,27,1,0,0,2,...,4,No Deposit,NaN,NaN,0,Personal,0.0,0,0,Check-Out
2,Resort Hotel,0,7,2017,September,27,1,0,1,1,...,0,No Deposit,NaN,NaN,0,Personal,75.0,0,0,Check-Out
3,Resort Hotel,0,13,2017,September,27,1,0,1,1,...,0,No Deposit,304.0,NaN,0,Personal,75.0,0,0,Check-Out
4,Resort Hotel,0,14,2017,September,27,1,0,2,2,...,0,No Deposit,240.0,NaN,0,Personal,98.0,0,1,Check-Out


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 29 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_weekdays_nights        119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

## Missing Values

In [18]:
#Check for missing values
mis_col = df.isna().sum()[df.isna().sum()>0]
mis_col

children         4
city           488
agent        16340
company     112593
dtype: int64

In [19]:
#Check missing columns data
for i in mis_col.index:
    print(f'Unique values in {i} column :',df[i].unique()[:4])

Unique values in children column : [ 0.  1.  2. 10.]
Unique values in city column : ['Kota Denpasar' 'Kabupaten Bangka' 'Kabupaten Sleman' 'Kota Batu']
Unique values in agent column : [ nan 304. 240. 303.]
Unique values in company column : [ nan 110. 113. 270.]


In [20]:
#Fill missing values
df[['children','agent','company']]= df[['children','agent','company']].fillna(0)
df['city'] = df['city'].fillna('unknown')

In [21]:
#Check missing values
df.isna().sum()[df.isna().sum()>0]

Series([], dtype: int64)

## Adjusting values

In [22]:
#check for categorical values
df['meal'].unique()

array(['Breakfast', 'Full Board', 'Dinner', 'No Meal', 'Undefined'],
      dtype=object)

In [23]:
#Correcting meal column values
def fx(x):
    if x == 'Undefined':
        return 'No Meal'
    else :
        return x
df['meal'] = df['meal'].map(lambda x: fx(x))
df['meal'].unique()

array(['Breakfast', 'Full Board', 'Dinner', 'No Meal'], dtype=object)

In [24]:
#Encoding Month values
months_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
               'September', 'October', 'November', 'December']
month_dict = {}

a = 1
for i in months_list:
    month_dict[i] = a
    a +=1
    
df['arrival_date_month'] = df['arrival_date_month'].map(month_dict)
df['arrival_date_month'].unique()

array([ 9, 10, 11, 12,  1,  2,  3,  4,  5,  6,  7,  8], dtype=int64)

In [25]:
#Convert data type to integer
df['children'] = df['children'].astype('int64')
df['agent'] = df['agent'].astype('int64')
df['company'] = df['company'].astype('int64')

## Drop Unnecesary Data

In [26]:
#Create new column total_guests
df['total_guests'] = df['adults'] + df['children'] + df['babies']
df['stay_nights'] = df['stays_in_weekend_nights'] + df['stays_in_weekdays_nights']

#filter data where total guest and stay night more than 0, 
#total_guests and stay night 0 means no one use the room and not spending night in the room
df = df[(df['total_guests']>0) & (df['stay_nights']>0)]
df.shape

(118565, 31)

In [ ]:
investigate_hotel_business_data_visualization